In [0]:
from google.colab import drive
drive.mount('/Drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /Drive


## Imports

In [0]:
!pip install git+https://www.github.com/keras-team/keras-contrib.git

  Cloning https://www.github.com/keras-team/keras-contrib.git to /tmp/pip-req-build-7qft7omg
  Running command git clone -q https://www.github.com/keras-team/keras-contrib.git /tmp/pip-req-build-7qft7omg
  Created wheel for keras-contrib: filename=keras_contrib-2.0.8-cp36-none-any.whl size=101066 sha256=4ffc8715e9466f1df130b30b3193afe520bf75b336d4420daf0d64f2767ef8a5
  Stored in directory: /tmp/pip-ephem-wheel-cache-srw5x8if/wheels/11/27/c8/4ed56de7b55f4f61244e2dc6ef3cdbaff2692527a2ce6502ba
Successfully built keras-contrib


In [0]:
from keras_contrib.layers.normalization.instancenormalization import InstanceNormalization
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, Concatenate, Lambda, Add
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.convolutional import UpSampling2D, Conv2D, Conv2DTranspose
from keras.layers.advanced_activations import LeakyReLU
from keras.activations import relu, tanh
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras.applications.resnet50 import ResNet50
from keras.layers import *
from keras.layers.merge import *
from keras import backend as K
from glob import glob
import matplotlib.pyplot as plt
import datetime
import scipy
import scipy.misc
import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim
import os
import sys
import time
import copy
import math
import cv2
import json

np.random.seed(42)


## Get Dataset

In [0]:
from tqdm import tqdm


GLASS_LABELS = dict()

def download():
  url = 'https://dl.dropboxusercontent.com/s/1i2h7orwm2cx8s0/UTKFace.tar.gz'
  fpath = 'UTKFace.tar.gz'
  filename = fpath.split('/')[-1]
  if not os.path.isfile(fpath):
    os.system(f'wget {url}')
    os.system(f'tar xzf {filename}')
  else:
    print("Data already present")
    

def copy_imgs_young(fpath, data_path):
  low = 18
  high = 25
  final_path_train = data_path + "/trainA"
  final_path_test = data_path + "/testA"
  if not os.path.isdir(final_path_train):
    os.makedirs(final_path_train)
  if not os.path.isdir(final_path_test):
    os.makedirs(final_path_test)
  allowed_files = []
  for fname in tqdm(glob(fpath+"/*.jpg")):
    age = int(fname.split('/')[-1].split('_')[0])
    if age >= low and age <= high:
      allowed_files.append(fname)
      
  allowed_files.sort()
  
  train_len = int(0.8*len(allowed_files))
  for fname in tqdm(allowed_files[:train_len]):
      os.system(f'cp {fname} {final_path_train}/')
  for fname in tqdm(allowed_files[train_len:]):
      os.system(f'cp {fname} {final_path_test}/')
      
def copy_imgs_old(fpath, data_path):
  low = 50
  high = 60
  final_path_train = data_path + "/trainB"
  final_path_test = data_path + "/testB"
  if not os.path.isdir(final_path_train):
    os.makedirs(final_path_train)
  if not os.path.isdir(final_path_test):
    os.makedirs(final_path_test)
  allowed_files = []
  for fname in tqdm(glob(fpath+"/*.jpg")):
    age = int(fname.split('/')[-1].split('_')[0])
    if age >= low and age <= high:
      allowed_files.append(fname)
  
  allowed_files.sort()
  
  train_len = int(0.8*len(allowed_files))
  for fname in tqdm(allowed_files[:train_len]):
      os.system(f'cp {fname} {final_path_train}/')
  for fname in tqdm(allowed_files[train_len:]):
      os.system(f'cp {fname} {final_path_test}/')
      
def fetch_glasses_labels():
  global GLASS_LABELS
  url = 'https://www.dl.dropboxusercontent.com/s/qryo4esy82er4ss/UTKFace_Glasses.json'
  os.system(f'wget {url}')
  d = json.load(open('UTKFace_Glasses.json'))
  paths = ['datasets/faces/trainA', 'datasets/faces/trainB']
  for folder in paths:
    for fpath in glob(folder+"/*.jpg"):
      fname = fpath.split("/")[-1]
      if fname in d:
        GLASS_LABELS[fpath] = d[fname]
  
def prepare_dataset():
  download()
  copy_imgs_young('UTKFace', 'datasets/faces')
  copy_imgs_old('UTKFace', 'datasets/faces')
#   fetch_glasses_labels()
  
  
prepare_dataset()

100%|██████████| 519/519 [00:05<00:00, 90.29it/s]


In [0]:
len(GLASS_LABELS)

3827

## Model declaration

In [0]:
class DataLoader():
  def __init__(self, dataset_name, img_res=(256, 256)):
    self.dataset_name = dataset_name
    self.img_res = img_res
    self.dataset_root_path = "datasets"

  def load_data(self, domain, batch_size=1, is_testing=False, img_res=(256, 256)):
    data_type = "train%s" % domain if not is_testing else "test%s" % domain
    path = glob('./datasets/%s/%s/*' % (self.dataset_name, data_type))

    batch_images = np.random.choice(path, size=batch_size)

    imgs = []
    labels = []
    for img_path in batch_images:
      img = self.imread(img_path)
      if not is_testing:
        img = cv2.resize(img, img_res)

        if np.random.random() > 0.5:
          img = np.fliplr(img)
      else:
        img = cv2.resize(img, img_res)
      imgs.append(img)
      labels.append(self.get_label(img_path))

    imgs = np.array(imgs)/127.5 - 1.

    return imgs, labels

  def load_batch(self, batch_size=1, is_testing=False, img_res=(256, 256)):
    data_type = "train" if not is_testing else "val"
    path_A = glob(f'./{self.dataset_root_path}/%s/%sA/*' % (self.dataset_name, data_type))
    path_B = glob(f'./{self.dataset_root_path}/%s/%sB/*' % (self.dataset_name, data_type))

    self.n_batches = int(min(len(path_A), len(path_B)) / batch_size)
    total_samples = self.n_batches * batch_size

    # Sample n_batches * batch_size from each path list so that model sees all
    # samples from both domains
    path_A = np.random.choice(path_A, total_samples, replace=False)
    path_B = np.random.choice(path_B, total_samples, replace=False)

    for i in range(self.n_batches-1):
      batch_A = path_A[i*batch_size:(i+1)*batch_size]
      batch_B = path_B[i*batch_size:(i+1)*batch_size]
      imgs_A, imgs_B = [], []
      labels_A, labels_B = [], []
      for img_A, img_B in zip(batch_A, batch_B):
        img_path_A = img_A
        img_path_B = img_B
        img_A = self.imread(img_A)
        img_B = self.imread(img_B)

        img_A = cv2.resize(img_A, img_res)
        img_B = cv2.resize(img_B, img_res)

        if not is_testing and np.random.random() > 0.5:
            img_A = np.fliplr(img_A)
            img_B = np.fliplr(img_B)

        imgs_A.append(img_A)
        imgs_B.append(img_B)
        labels_A.append(self.get_label(img_path_A))
        labels_B.append(self.get_label(img_path_B))

      imgs_A = np.array(imgs_A)/127.5 - 1.
      imgs_B = np.array(imgs_B)/127.5 - 1.

      yield imgs_A, imgs_B, labels_A, labels_B
  
  def get_label(self, img_path):
    gender = int(img_path.split('/')[-1].split('_')[1])
    if gender == 0:  # Male
      return 1
    elif gender == 1:  # Female
      return 2

  def load_img(self, path, img_res=(256, 256)):
    img = self.imread(path)
    img = cv2.resize(img, img_res)
    img = img/127.5 - 1.
    return img[np.newaxis, :, :, :]

  def imread(self, path):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB).astype(np.float)
    return img

In [0]:
class CycleGAN():
    def __init__(self, img_rows, img_cols, channels, save_path='/Drive/My Drive/facegan/train',
                 output_img_path='images', c_path=None):
        # Input shape
        self.img_rows = img_rows
        self.img_cols = img_cols
        self.channels = channels
        self.img_shape = (self.img_rows, self.img_rows, self.channels)

        # Configure data loader
        self.dataset_name = 'faces'
        self.data_loader = DataLoader(dataset_name=self.dataset_name,
                                      img_res=(self.img_rows, self.img_cols))

        self.save_path = save_path
        self.output_img_path = 'images'

        # Calculate output shape of D (PatchGAN)
        patch = int(self.img_rows / 2 ** 4)
        patch = int(self.img_rows / 2 ** 4)
        patch *= 2  # hack to get the shape 32,32,1. todo: check
        self.disc_patch = (patch, patch, 1)

        # Number of filters in the first layer of G and D
        self.gf = 64
        self.df = 64

        # Loss weights
        self.lambda_cycle = 10.0  # Cycle-consistency loss
        self.lambda_id = 0.1 * self.lambda_cycle  # Identity loss

        self.lr_init = 0.0002
        self.epoch_step = 100

        # InfoGAN classes - currently gender
        self.num_classes = 2

        optimizer = Adam(self.lr_init, 0.5)

        # Build and compile the discriminators
        self.d_A, self.d_Labels_A = self.build_discriminators()
        self.d_B, self.d_Labels_B = self.build_discriminators()

        print("Summary Discriminator - Labels:")
        print(self.d_Labels_A.summary())
        
        self.d_A.compile(loss='mse',
                         optimizer=optimizer,
                         metrics=['accuracy'])
        self.d_B.compile(loss='mse',
                         optimizer=optimizer,
                         metrics=['accuracy'])
        # Info-GAN
        self.d_Labels_A.compile(loss=[self.mutual_info_loss],
                                optimizer=optimizer,
                                metrics=['accuracy'])
        self.d_Labels_B.compile(loss=[self.mutual_info_loss],
                                optimizer=optimizer,
                                metrics=['accuracy'])

        # -------------------------
        # Construct Computational
        #   Graph of Generators
        # -------------------------

        # Build the generators
        self.g_AB = self.build_generator()
        self.g_BA = self.build_generator()

        print("Summary Generator:")
        print(self.g_AB.summary())

        # Input images from both domains
        # img_A = Input(shape=self.img_shape)
        # img_B = Input(shape=self.img_shape)
        img_A = Input(shape=(self.img_shape))
        img_B = Input(shape=(self.img_shape))
        label_A_inp = Input(shape=(1,), dtype='int32')
        label_B_inp = Input(shape=(1,), dtype='int32')

        # Translate images to the other domain
        fake_B = self.g_AB([img_A, label_A_inp])
        fake_A = self.g_BA([img_B, label_B_inp])
        # Translate images back to original domain
        reconstr_A = self.g_BA([fake_B, label_B_inp])
        reconstr_B = self.g_AB([fake_A, label_A_inp])
        # Identity mapping of images
        img_A_id = self.g_BA([img_A, label_A_inp])
        img_B_id = self.g_AB([img_B, label_B_inp])

        # For the combined model we will only train the generators
        self.d_A.trainable = False
        self.d_B.trainable = False

        # Discriminators determines validity of translated images
        valid_A = self.d_A(fake_A)
        valid_B = self.d_B(fake_B)

        gan_outputs = [valid_A, valid_B,
                       reconstr_A, reconstr_B,
                       img_A_id, img_B_id]

        gan_losses = ['mse', 'mse',
                      'mae', 'mae',
                      'mae', 'mae']
        gan_loss_weights = [1, 1,
                            self.lambda_cycle, self.lambda_cycle,
                            self.lambda_id, self.lambda_id]

        # ****** PERCEPT ****
        #     self.model_perceptual = self.build_perceptual_model(input_shape=self.img_shape)

        #     percept_A = self.model_perceptual([img_A])
        #     percept_B = self.model_perceptual([img_B])
        #     percept_reconstr_A = self.model_perceptual([reconstr_A])
        #     percept_reconstr_B = self.model_perceptual([reconstr_B])
        #     gan_outputs.extend([percept_A, percept_B,
        #                        percept_reconstr_A, percept_reconstr_B,
        #                        percept_reconstr_A, percept_reconstr_B])

        #     gan_losses.extend(['mse', 'mse',
        #                 'mse', 'mse',
        #                 'mse', 'mse'])
        #     gan_loss_weights.extend([self.lambda_id, self.lambda_id,
        #                         self.lambda_id, self.lambda_id,
        #                         self.lambda_id, self.lambda_id])
        # ****** PERCEPT ****
        label_A = self.d_Labels_A(fake_A)
        label_B = self.d_Labels_B(fake_B)
        label_A_reconstr = self.d_Labels_A(reconstr_A)
        label_B_reconstr = self.d_Labels_B(reconstr_B)
        gan_outputs.extend([
            label_A, label_B,
            label_A_reconstr, label_B_reconstr,
            label_A_reconstr, label_B_reconstr
        ])
        gan_losses.extend([self.mutual_info_loss, self.mutual_info_loss,
                           self.mutual_info_loss, self.mutual_info_loss,
                           self.mutual_info_loss, self.mutual_info_loss])
        gan_loss_weights.extend([self.lambda_id, self.lambda_id,
                                 self.lambda_id, self.lambda_id,
                                 self.lambda_id, self.lambda_id])

        # Combined model trains generators to fool discriminators
        self.combined = Model(inputs=[img_A, label_A_inp, img_B, label_B_inp],
                              outputs=gan_outputs)
        if c_path is not None:
            print(f"Loading weights: {c_path}")
            self.combined.load_weights(c_path)
            
            
        self.combined.compile(loss=gan_losses,
                              loss_weights=gan_loss_weights,
                              optimizer=optimizer)

    def mutual_info_loss(self, c, c_given_x):
        """The mutual information metric we aim to minimize (From InfoGAN)"""
        eps = 1e-8
        conditional_entropy = K.mean(- K.sum(K.log(c_given_x + eps) * c, axis=1))
        entropy = K.mean(- K.sum(K.log(c + eps) * c, axis=1))

        return conditional_entropy + entropy

    def build_generator(self, name="generator"):
        gf_dim = self.gf
        output_c_dim = self.channels

        def conv2D(input_,
                   output_dim,
                   ks=4,
                   s=2,
                   stddev=0.02,
                   padding='SAME',
                   activation=False,
                   instance_norm=False,
                   name="conv2d",
                   act_name="relu",
                   activation_tanh=True,
                   inst_norm_name="bn"):

            if activation_tanh:
                x = Conv2D(
                    output_dim,
                    kernel_size=(ks, ks),
                    strides=s,
                    padding=padding,
                    kernel_initializer=tf.truncated_normal_initializer(stddev=stddev),
                    name=name,
                    activation='tanh',
                    use_bias=False)(input_)
            elif activation:
                x = Conv2D(
                    output_dim,
                    kernel_size=(ks, ks),
                    strides=s,
                    padding=padding,
                    kernel_initializer=tf.truncated_normal_initializer(stddev=stddev),
                    name=name,
                    activation='relu',
                    use_bias=False)(input_)
            else:
                x = Conv2D(
                    output_dim,
                    kernel_size=(ks, ks),
                    strides=s,
                    padding=padding,
                    kernel_initializer=tf.truncated_normal_initializer(stddev=stddev),
                    name=name,
                    use_bias=False)(input_)
            if instance_norm:
                x = InstanceNormalization(name=inst_norm_name)(x)

            return x

        def deconv2d(input_, output_dim, ks=4, s=2, stddev=0.02, name="deconv2d"):
            x = Conv2DTranspose(
                output_dim,
                kernel_size=(ks, ks),
                strides=s,
                padding='SAME',
                kernel_initializer=tf.truncated_normal_initializer(stddev=stddev),
                name=name,
                use_bias=False)(input_)

            return x

        def residule_block(x, dim, ks=3, s=1, name='res'):
            p = int((ks - 1) / 2)
            y = Lambda(lambda lx: tf.pad(lx, [[0, 0], [p, p], [p, p], [0, 0]], "REFLECT"))(x)
            y = conv2D(y, dim, ks=ks, s=s, padding="VALID", name=name + "_c1", activation=True, instance_norm=True,
                       inst_norm_name=name + "_bn1")
            y = Lambda(lambda x: tf.pad(x, [[0, 0], [p, p], [p, p], [0, 0]], "REFLECT"))(y)
            y = conv2D(y, dim, ks=ks, s=s, padding="VALID", name=name + "_c2", activation=False, instance_norm=True,
                       inst_norm_name=name + "_bn2")
            a = Add()([y, x])
            return a  # 1/2을 안해도 되나??? 뒤에 노말라이즈해서 상관없는건가....

        # Justin Johnson's model from https://github.com/jcjohnson/fast-neural-style/
        # The network with 9 blocks consists of: c7s1-32, d64, d128, R128, R128, R128,
        # R128, R128, R128, R128, R128, R128, u64, u32, c7s1-3

        # inp = Input(shape=self.img_shape)
        
        img_inp = Input(shape=(self.img_shape))
        inp_label = Input(shape=(1,), dtype='int32')
       
        label_embedding = Flatten()(Embedding(num_classes, (self.img_rows*self.img_cols*self.channels))(inp_label))
        label_embedding = Reshape((self.img_rows,self.img_cols,self.channels))(label_embedding)

        # inp = Multiply()([img_inp, label_embedding])
        inp = multiply([img_inp, label_embedding])
        

        c0 = Lambda(lambda x: tf.pad(x, [[0, 0], [3, 3], [3, 3], [0, 0]], "REFLECT"))(inp)

        c1 = conv2D(c0, gf_dim, ks=7, s=1, padding="VALID", name="g_e1_c", activation=True, instance_norm=True,
                    inst_norm_name=name + "g_e1_bn", act_name="g_e1_relu")
        c2 = conv2D(c1, gf_dim * 2, ks=3, s=2, name="g_e2_c", activation=True, instance_norm=True,
                    inst_norm_name=name + "g_e2_bn", act_name="g_e2_relu")
        c3 = conv2D(c2, gf_dim * 4, ks=3, s=2, name="g_e3_c", activation=True, instance_norm=True,
                    inst_norm_name=name + "g_e3_bn", act_name="g_e3_relu")  # 64x64

        # define G network with 9 resnet blocks
        # 총 cnn 18층.
        r1 = residule_block(c3, gf_dim * 4, name='g_r1')
        r2 = residule_block(r1, gf_dim * 4, name='g_r2')
        r3 = residule_block(r2, gf_dim * 4, name='g_r3')
        r4 = residule_block(r3, gf_dim * 4, name='g_r4')
        r5 = residule_block(r4, gf_dim * 4, name='g_r5')
        r6 = residule_block(r5, gf_dim * 4, name='g_r6')
        r7 = residule_block(r6, gf_dim * 4, name='g_r7')
        r8 = residule_block(r7, gf_dim * 4, name='g_r8')
        r9 = residule_block(r8, gf_dim * 4, name='g_r9')

        d1 = deconv2d(r9, gf_dim * 2, 3, 2, name='g_d1_dc')
        d1 = InstanceNormalization(name='g_d1_bn')(d1)
        d1 = Lambda(lambda x: relu(x))(d1)

        d2 = deconv2d(d1, gf_dim, 3, 2, name='g_d2_dc')
        d2 = InstanceNormalization(name='g_d2_bn')(d2)
        d2 = Lambda(lambda x: relu(x))(d2)

        d2 = Lambda(lambda x: tf.pad(x, [[0, 0], [3, 3], [3, 3], [0, 0]], "REFLECT"))(d2)

        pred = conv2D(d2, output_c_dim, ks=7, s=1, padding="VALID", name="g_pred_c", activation=False,
                      instance_norm=False, activation_tanh=True)
        
        return Model([img_inp, inp_label], pred)

    def build_perceptual_model(self, input_shape, trainable=False, pop=True):
        # import ResNet50 pretrained on imagenet
        model = ResNet50(include_top=False, weights='imagenet', input_shape=input_shape)
        if pop == True:
            model.layers.pop()  # pop pooling layer
            model.layers.pop()  # pop last activation layer

        for layer in model.layers:
            layer.trainable = trainable

        #         print('Resnet50 for Perceptual loss:')
        #         model.summary()
        return model

    def build_discriminators(self, name="discriminator"):
        df_dim = self.df

        def conv2D(input_,
                   output_dim,
                   ks=4,
                   s=2,
                   stddev=0.02,
                   padding='SAME',
                   activation=False,
                   instance_norm=False,
                   name="conv2d",
                   act_name="lrelu",
                   inst_norm_name="bn"):

            x = Conv2D(
                output_dim,
                kernel_size=(ks, ks),
                strides=s,
                padding=padding,
                kernel_initializer=tf.truncated_normal_initializer(stddev=stddev),
                name=name,
                use_bias=False)(input_)
            if activation:
                x = LeakyReLU(alpha=0.2, name=act_name)(x)
            if instance_norm:
                x = InstanceNormalization(name=inst_norm_name)(x)

            return x

        inp = Input(shape=self.img_shape)
        h0 = conv2D(inp, df_dim, name='d_h0_conv', activation=True)
        # h0 = lrelu(conv2d(image, df_dim, name='d_h0_conv'))
        # h0 is (128 x 128 x self.df_dim)

        h1 = conv2D(h0, df_dim * 2, name='d_h1_conv', activation=True, instance_norm=True, inst_norm_name='d_bn1',
                    act_name='d_lrelu1')
        # h1 = lrelu(instance_norm(conv2d(h0, df_dim*2, name='d_h1_conv'), 'd_bn1'))
        # h1 is (64 x 64 x self.df_dim*2)

        h2 = conv2D(h1, df_dim * 4, name='d_h2_conv', activation=True, instance_norm=True, inst_norm_name='d_bn2',
                    act_name='d_lrelu2')
        # h2 = lrelu(instance_norm(conv2d(h1, df_dim*4, name='d_h2_conv'), 'd_bn2'))
        # h2 is (32x 32 x self.df_dim*4)

        h3 = conv2D(h2, df_dim * 8, name='d_h3_conv', s=1, activation=True, instance_norm=True, inst_norm_name='d_bn3',
                    act_name='d_lrelu3')
        # h3 = lrelu(instance_norm(conv2d(h2, df_dim*8, s=1, name='d_h3_conv'), 'd_bn3'))
        # h3 is (32 x 32 x self.df_dim*8)

        h4 = conv2D(h3, 1, name='d_h3_pred', s=1, activation=False, instance_norm=False)
        # h4 = conv2d(h3, 1, s=1, name='d_h3_pred')
        # h4 is (32 x 32 x 1)

        # Recognition
        print("h4", h4.shape)
        label = Dense(128, activation='relu')(h4)
        label = GlobalAveragePooling2D()(label)
        print("lab", label.shape)
        label = Dense(self.num_classes, activation='softmax')(label)
        print("lab", label.shape)

        return Model(inp, h4), Model(inp, label)

    def train(self, epochs, batch_size=1, sample_interval=500, init_epoch=0):

        start_time = datetime.datetime.now()

        # Adversarial loss ground truths
        valid = np.ones((batch_size,) + self.disc_patch)
        fake = np.zeros((batch_size,) + self.disc_patch)

        for epoch in range(init_epoch, epochs):
            lr = self.lr_init if epoch < self.epoch_step else self.lr_init * (self.epoch - epoch) / (
                    self.epoch - self.epoch_step)  # lr이 작아짐. 100에폭이 지나면.
            for batch_i, (imgs_A, imgs_B, labels_A, labels_B) in enumerate(
                    self.data_loader.load_batch(batch_size, img_res=(self.img_rows, self.img_cols))):

                K.set_value(self.d_A.optimizer.lr, lr)
                K.set_value(self.d_B.optimizer.lr, lr)
                K.set_value(self.combined.optimizer.lr, lr)

                # ----------------------
                #  Train Discriminators
                # ----------------------

                # Translate images to opposite domain
                labels_A = np.array(labels_A).reshape(-1, 1)
                labels_B = np.array(labels_B).reshape(-1, 1)
                imgs_A_gen_input = [imgs_A, labels_A]
                imgs_B_gen_input = [imgs_B, labels_B]

                fake_B = self.g_AB.predict(imgs_A_gen_input)
                fake_A = self.g_BA.predict(imgs_B_gen_input)

                # Train the discriminators (original images = real / translated = Fake)
                dA_loss_real = self.d_A.train_on_batch(imgs_A, valid)
                dA_loss_fake = self.d_A.train_on_batch(fake_A, fake)
                dA_loss = 0.5 * np.add(dA_loss_real, dA_loss_fake)

                dB_loss_real = self.d_B.train_on_batch(imgs_B, valid)
                dB_loss_fake = self.d_B.train_on_batch(fake_B, fake)
                dB_loss = 0.5 * np.add(dB_loss_real, dB_loss_fake)

                # Total disciminator loss
                d_loss = 0.5 * np.add(dA_loss, dB_loss)

                # ------------------
                #  Train Generators
                # ------------------

                gan_train_outputs = [valid, valid,
                                     imgs_A, imgs_B,
                                     imgs_A, imgs_B]

                # ****** PERCEPT ****
                #         percept_A_train = self.model_perceptual.predict(imgs_A)
                #         percept_B_train = self.model_perceptual.predict(imgs_B)
                #         percept_fake_A_train = self.model_perceptual.predict(fake_A)
                #         percept_fake_B_train = self.model_perceptual.predict(fake_B)
                #         gan_train_outputs.extend([percept_fake_B_train, percept_fake_A_train,
                #                        percept_fake_A_train, percept_fake_B_train,
                #                        percept_A_train, percept_B_train])
                # ****** PERCEPT ****
                label_A = self.d_Labels_A.predict(imgs_A)
                label_B = self.d_Labels_B.predict(imgs_B)
                label_A_reconstr = self.d_Labels_A.predict(fake_A)
                label_B_reconstr = self.d_Labels_B.predict(fake_B)
                gan_train_outputs.extend([label_B_reconstr, label_A_reconstr,
                                          label_A_reconstr, label_B_reconstr,
                                          label_A, label_B])

                # Train the generators

                g_loss = self.combined.train_on_batch([imgs_A_gen_input[0],imgs_A_gen_input[1], imgs_B_gen_input[0], imgs_B_gen_input[1]],
                                                      gan_train_outputs)

                elapsed_time = datetime.datetime.now() - start_time

                loss_index = 6

                if batch_i % 100 == 0:
                    # Plot the progress
                    #           print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %05f, adv: %05f, recon: %05f, id: %05f] time: %s " \
                    #                                       % ( epoch, epochs,
                    #                                         batch_i, self.data_loader.n_batches,
                    #                                         d_loss[0], 100*d_loss[1],
                    #                                         g_loss[0],
                    #                                         np.mean(g_loss[1:3]),
                    #                                         np.mean(g_loss[3:5]),
                    #                                         np.mean(g_loss[5:6]),
                    #                                         elapsed_time))
                    # ****** PERCEPT ****
                    print(
                        "[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %3d%%] [G loss: %05f, adv: %05f, recon: %05f, id: %05f, per_fB: %05f, per_fA: %05f, per_fArB: %05f, per_fBrA: %05f, per_rBrA: %05f, per_rBrB: %05f] time: %s " \
                        % (epoch, epochs,
                           batch_i, self.data_loader.n_batches,
                           d_loss[0], 100 * d_loss[1],
                           g_loss[0],
                           np.mean(g_loss[1:3]),
                           np.mean(g_loss[3:5]),
                           np.mean(g_loss[5:6]),
                           np.mean(g_loss[6]),
                           np.mean(g_loss[7]),
                           np.mean(g_loss[8]),
                           np.mean(g_loss[9]),
                           np.mean(g_loss[10]),
                           np.mean(g_loss[11]),
                           elapsed_time))
                    # ****** PERCEPT ****

                # If at save interval => save generated image samples
                if batch_i % sample_interval == 0:
                    self.sample_images(epoch, batch_i)
                    self.save_model(epoch)

    def save_model(self, epoch):
        print("Saving model now")
        if not os.path.isdir(self.save_path):
            os.makedirs(self.save_path)
        # for further training - save with
        self.combined.save_weights(self.save_path + '/%s-%d.h5' % (self.dataset_name, epoch))

        # reload with
        # self.combined.load_weights('train/%s-%d.h5' % (self.dataset_name, epoch))

        # to generate images - save with
        self.g_AB.save_weights(self.save_path + '/%s-AB-%d.h5' % (self.dataset_name, epoch))
        self.g_BA.save_weights(self.save_path + '/%s-BA-%d.h5' % (self.dataset_name, epoch))

        # load with
        # self.g_AB.load_weights('train/%s-AB-%d.h5' % (self.dataset_name, epoch))
        # self.g_BA.load_weights('train/%s-BA-%d.h5' % (self.dataset_name, epoch))

    def parse_predicted_labels(self, labels):
      parsed = []
      for i, d in enumerate(labels):
        parsed.append(np.argmax(d) + 1)
      
      return np.array(parsed)
      
    def sample_images(self, epoch, batch_i):
        os.makedirs(self.output_img_path + '/%s' % self.dataset_name, exist_ok=True)
        r, c = 2, 3

        imgs_A, labels_A = self.data_loader.load_data(domain="A", batch_size=1, is_testing=True,
                                            img_res=(self.img_rows, self.img_cols))
        imgs_B, labels_B = self.data_loader.load_data(domain="B", batch_size=1, is_testing=True,
                                            img_res=(self.img_rows, self.img_cols))

        # Demo (for GIF)
        # imgs_A = self.data_loader.load_img('datasets/apple2orange/testA/n07740461_1541.jpg')
        # imgs_B = self.data_loader.load_img('datasets/apple2orange/testB/n07749192_4241.jpg')

        # Translate images to the other domain
        labels_A = np.array(labels_A).reshape(-1, 1)
        labels_B = np.array(labels_B).reshape(-1, 1)
        imgs_A_gen_input = [imgs_A, labels_A]
        imgs_B_gen_input = [imgs_B, labels_B]
        
        fake_B = self.g_AB.predict(imgs_A_gen_input)
        fake_A = self.g_BA.predict(imgs_B_gen_input)
        fake_labels_A = self.d_Labels_A.predict(fake_A)
        fake_labels_B = self.d_Labels_B.predict(fake_B)
        fake_labels_A = self.parse_predicted_labels(fake_labels_A).reshape(-1,1)
        fake_labels_B = self.parse_predicted_labels(fake_labels_A).reshape(-1,1)
        
        print("fake_labels_B.shape", fake_labels_B.shape)
        print("fake_labels", fake_labels_B)
        
        # Translate back to original domain
        fake_gen_input_A = [fake_A, fake_labels_A]
        fake_gen_input_B = [fake_B, fake_labels_B]
        reconstr_A = self.g_BA.predict(fake_gen_input_B)
        reconstr_B = self.g_AB.predict(fake_gen_input_A)

        gen_imgs = np.concatenate([imgs_A, fake_B, reconstr_A, imgs_B, fake_A, reconstr_B])

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        titles = ['Original', 'Translated', 'Reconstructed']
        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i, j].imshow(gen_imgs[cnt])
                axs[i, j].set_title(titles[j])
                axs[i, j].axis('off')
                cnt += 1
        fig.savefig(self.output_img_path + "/%s/%d_%d.png" % (self.dataset_name, epoch, batch_i))
        plt.close()


## Training

In [0]:
gan = CycleGAN(img_rows=128,img_cols=128,channels=3,save_path='/Drive/My Drive/facegan/train',output_img_path='images',c_path=None)
gan.train(1, init_epoch=0)